In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
voc=pd.read_csv('./data/voc_2010-2022.csv')
voc['Datetime']=pd.to_datetime(voc['Datetime'])
heat=pd.read_csv('./data/sensible_heat_flux_2001-2022.csv')
heat['Datetime']=pd.to_datetime(heat['Datetime'])
latent2018=pd.read_csv('./data/latent_heat_flux_2018-2022.csv')
latent2001=pd.read_csv('./data/latent_heat_flux_2001-2018.csv')
gpp=pd.read_csv('./data/GPP_NEE_flux_2001-2022.csv')
latent2001['Datetime']=pd.to_datetime(latent2001['Datetime'])
latent2018['Datetime']=pd.to_datetime(latent2018['Datetime'])
gpp['Datetime']=pd.to_datetime(gpp['Datetime'])

In [2]:
def closest(df1,df2,delta):
    # Funktio joka palauttaa df2:n aikoja edeltävät df1:n ajan ja lukuarvot
    # Delta on mahdollinen ajansiirto jos halutaan katsoa havaintoja tietyn ajan ennen df2:n aikoja
    curr=0 
    close=[]
    names=list(df1.columns)
    for val in df2['Datetime']:
        temp=0
        for j in range(curr,len(df1['Datetime'])):
            if df1['Datetime'][j]-delta<=val:
                temp=df1['Datetime'][j]
            elif temp==0:
                close.append([np.NaN,np.NaN])
                curr=j
                break
            else:
                app=np.asarray(df1.loc[j-1,names[1:]])
                close.append(app)
                curr=j
                break
    names=names[2:]
    names.insert(0,f'Closest time for {names[0]} etc')
    return close,names

In [3]:
from datetime import timedelta
delta=timedelta(0)
close,names=closest(heat,gpp,delta)

In [4]:
closedf=pd.DataFrame(close,columns=names)
combined=pd.concat([gpp,closedf],axis=1)

In [5]:
close,names=closest(latent2018,combined,delta)

In [6]:
closedf=pd.DataFrame(close,columns=names)
combined=pd.concat([combined,closedf],axis=1)

In [7]:
close,names=closest(latent2001,combined,delta)

In [8]:
closedf=pd.DataFrame(close,columns=names)
combined=pd.concat([combined,closedf],axis=1)

In [9]:
close,names=closest(voc,combined,delta)

In [10]:
closedf=pd.DataFrame(close,columns=names)
combined=pd.concat([combined,closedf],axis=1)

In [11]:
combined.head()

,Unnamed: 0,Datetime,NEE,NEE_gapfilling_method,GPP,Closest time for sensible_heat_flux etc,sensible_heat_flux,sensible_heat_flux_gapfilling_method,Closest time for latent_heat_flux_after_4-2018 etc,latent_heat_flux_after_4-2018,latent_heat_flux_quality_flag_after_4-2018,Closest time for HYY_EDDY233.LE etc,HYY_EDDY233.LE,HYY_EDDY233.Qc_LE,Closest time for monoterpene_flux etc,monoterpene_flux,methanol_flux,acetone_flux,acetaldehyde_flux
0,0,2001-01-01 00:00:00,0.459,1,-0.0,2001-01-01 00:00:00,2.175,0.0,NaT,NaN,NaN,2001-01-01 00:00:00,1.41180,NaN,NaT,NaN,NaN,NaN,NaN
1,1,2001-01-01 00:30:00,0.459,1,-0.0,2001-01-01 00:30:00,3.724,0.0,NaT,NaN,NaN,2001-01-01 00:30:00,1.19590,NaN,NaT,NaN,NaN,NaN,NaN
2,2,2001-01-01 01:00:00,0.459,1,-0.0,2001-01-01 01:00:00,8.380,0.0,NaT,NaN,NaN,2001-01-01 01:00:00,1.16250,NaN,NaT,NaN,NaN,NaN,NaN
3,3,2001-01-01 01:30:00,0.459,1,-0.0,2001-01-01 01:30:00,-3.257,0.0,NaT,NaN,NaN,2001-01-01 01:30:00,0.10336,NaN,NaT,NaN,NaN,NaN,NaN
4,4,2001-01-01 02:00:00,0.458,1,-0.0,2001-01-01 02:00:00,18.369,0.0,NaT,NaN,NaN,2001-01-01 02:00:00,0.20723,NaN,NaT,NaN,NaN,NaN,NaN


In [12]:
voc.isna().sum()

Unnamed: 0              0
Datetime                0
monoterpene_flux     1050
methanol_flux         644
acetone_flux          479
acetaldehyde_flux    3193
dtype: int64

In [13]:
latent2018.isna().sum()

Unnamed: 0                                    0
Datetime                                      0
latent_heat_flux_after_4-2018                 0
latent_heat_flux_quality_flag_after_4-2018    0
dtype: int64

In [14]:
latent2001.isna().sum()

Unnamed: 0                0
Datetime                  0
HYY_EDDY233.LE         2642
HYY_EDDY233.Qc_LE    238505
dtype: int64

In [15]:
heat.isna().sum()

Unnamed: 0                              0
Datetime                                0
sensible_heat_flux                      0
sensible_heat_flux_gapfilling_method    0
dtype: int64

In [16]:
gpp.isna().sum()

Unnamed: 0               0
Datetime                 0
NEE                      0
NEE_gapfilling_method    0
GPP                      0
dtype: int64

In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
# Täyttö koko datan keskiarvolla keskiarvolla
simple= SimpleImputer(missing_values=np.NaN, strategy='mean')
simple.fit(np.asarray(latent2001['HYY_EDDY233.LE']).reshape(-1, 1))
latent2001['simple HYY_EDDY233.LE']=simple.transform(np.asarray(latent2001['HYY_EDDY233.LE']).reshape(-1, 1))

simple.fit(np.asarray(voc['monoterpene_flux']).reshape(-1, 1))
voc['simple monoterpene_flux']=simple.transform(np.asarray(voc['monoterpene_flux']).reshape(-1, 1))

simple.fit(np.asarray(voc['methanol_flux']).reshape(-1, 1))
voc['simple methanol_flux']=simple.transform(np.asarray(voc['methanol_flux']).reshape(-1, 1))

simple.fit(np.asarray(voc['acetone_flux']).reshape(-1, 1))
voc['simple acetone_flux']=simple.transform(np.asarray(voc['acetone_flux']).reshape(-1, 1))

simple.fit(np.asarray(voc['acetaldehyde_flux']).reshape(-1, 1))
voc['simple acetaldehyde_flux']=simple.transform(np.asarray(voc['acetaldehyde_flux']).reshape(-1, 1))

In [18]:
# Iteratiivinen täyttö
iter=IterativeImputer(max_iter=10, random_state=0)
iter.fit(voc[['monoterpene_flux','methanol_flux','acetone_flux','acetaldehyde_flux']])
voc[['iter monoterpene_flux','iter methanol_flux','iter acetone_flux','iter acetaldehyde_flux']]=iter.transform(voc[['monoterpene_flux','methanol_flux','acetone_flux','acetaldehyde_flux']])

In [19]:
# Täyttö knn:llä
knn= KNNImputer(n_neighbors=5, weights="distance", missing_values=np.NaN)

knn.fit(voc[['monoterpene_flux','methanol_flux','acetone_flux','acetaldehyde_flux']])
voc[['knn monoterpene_flux','knn methanol_flux','knn acetone_flux','knn acetaldehyde_flux']]=knn.transform(voc[['monoterpene_flux','methanol_flux','acetone_flux','acetaldehyde_flux']])


In [36]:

daily_voc = voc.resample('D', on = 'Datetime').mean().reset_index()
daily_latent2001 = latent2001.resample('D', on = 'Datetime').mean().reset_index()

In [37]:
#Täyttö päivittäisillä keskiarvoilla

# Create a copy of the dataframe to avoid modifying the original
daily_voc_years = daily_voc.copy()

# Add a column for the day of year
daily_voc_years['day'] = daily_voc_years['Datetime'].dt.dayofyear

# Add a column for the year
daily_voc_years['year'] = daily_voc_years['Datetime'].dt.year

# Drop the column 'Datetime'
daily_voc_years = daily_voc_years.drop(columns='Datetime')

# Create a copy of the dataframe to avoid modifying the original
daily_latent2001_years = daily_latent2001.copy()

# Add a column for the day of year
daily_latent2001_years['day'] = daily_latent2001_years['Datetime'].dt.dayofyear

# Add a column for the year
daily_latent2001_years['year'] = daily_latent2001_years['Datetime'].dt.year

# Drop the column 'Datetime'
daily_latent2001_years = daily_latent2001_years.drop(columns='Datetime')

# Pivot the DataFrame so that rows are weeks and columns are years
daily_methanol_years = daily_voc_years.pivot(index='day', columns='year', values='methanol_flux')
daily_monoterpene_years = daily_voc_years.pivot(index='day', columns='year', values='monoterpene_flux')
daily_acetone_years = daily_voc_years.pivot(index='day', columns='year', values='acetone_flux')
daily_acetaldehyde_years = daily_voc_years.pivot(index='day', columns='year', values='acetaldehyde_flux')
daily_latent2001_years = daily_latent2001_years.pivot(index='day', columns='year', values='HYY_EDDY233.LE')



In [39]:

daily_methanol_years_fill=daily_methanol_years.copy()
daily_methanol_years_fill=daily_methanol_years_fill.apply(lambda row: row.fillna(row.mean()), axis=1)

daily_monoterpene_years_fill=daily_monoterpene_years.copy()
daily_monoterpene_years_fill=daily_monoterpene_years_fill.apply(lambda row: row.fillna(row.mean()), axis=1)

daily_acetone_years_fill=daily_acetone_years.copy()
daily_acetone_years_fill=daily_acetone_years_fill.apply(lambda row: row.fillna(row.mean()), axis=1)

daily_acetaldehyde_years_fill=daily_acetaldehyde_years.copy()
daily_acetaldehyde_years_fill=daily_acetaldehyde_years_fill.apply(lambda row: row.fillna(row.mean()), axis=1)

daily_latent2001_years_fill=daily_latent2001_years.copy()
daily_latent2001_years_fill=daily_latent2001_years_fill.apply(lambda row: row.fillna(row.mean()), axis=1)